In [1]:
%env HF_ENDPOINT=https://hf-mirror.com

env: HF_ENDPOINT=https://hf-mirror.com


In [2]:
import argparse
import os

import mindspore
from mindspore.experimental.optim import AdamW
from tqdm import tqdm
import evaluate
from mindnlp.dataset import load_dataset
from mindnlp.engine import set_seed
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from mindnlp.modules.optimization import get_linear_schedule_with_warmup
from mindnlp.peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PromptTuningConfig,
)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.612 seconds.
Prefix dict has been built successfully.


In [3]:
batch_size = 32
model_name_or_path = "roberta-large"
task = "mrpc"
peft_type = PeftType.PROMPT_TUNING
num_epochs = 20

In [4]:
peft_config = PromptTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=10)
lr = 1e-3

In [5]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [6]:
datasets = load_dataset("glue", task)
print(next(datasets['train'].create_dict_iterator()))

{'sentence1': Tensor(shape=[], dtype=String, value= 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'), 'sentence2': Tensor(shape=[], dtype=String, value= 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'), 'label': Tensor(shape=[], dtype=Int64, value= 1), 'idx': Tensor(shape=[], dtype=Int64, value= 0)}


In [7]:
from mindnlp.dataset import BaseMapFuction

class MapFunc(BaseMapFuction):
    def __call__(self, sentence1, sentence2, label, idx):
        outputs = tokenizer(sentence1, sentence2, truncation=True, max_length=None)
        return outputs['input_ids'], outputs['attention_mask'], label


def get_dataset(dataset, tokenizer):
    input_colums=['sentence1', 'sentence2', 'label', 'idx']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})
    return dataset

train_dataset = get_dataset(datasets['train'], tokenizer)
eval_dataset = get_dataset(datasets['validation'], tokenizer)

In [8]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[32, 70], dtype=Int64, value=
[[    0, 10127,  1001 ...     1,     1,     1],
 [    0,   975, 26802 ...     1,     1,     1],
 [    0,  1213,    56 ...     1,     1,     1],
 ...
 [    0,   133,  1154 ...     1,     1,     1],
 [    0, 12667,  8423 ...     1,     1,     1],
 [    0, 32478,  1033 ...     1,     1,     1]]), 'attention_mask': Tensor(shape=[32, 70], dtype=Int64, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 ...
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), 'labels': Tensor(shape=[32], dtype=Int64, value= [1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 
 1, 1, 0, 0, 1, 1, 1, 0])}


In [9]:
metric = evaluate.load("glue", task)

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

The following parameters in checkpoint files are not loaded:
['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.embeddings.position_ids']
The following parameters in models are missing parameter:
['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']


trainable params: 2,113,540 || all params: 356,423,684 || trainable%: 0.5929852854559463


In [11]:
optimizer = AdamW(params=model.trainable_params(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataset) * num_epochs),
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [12]:
model.trainable_params()

[Parameter (Tensor(shape=[1024, 1024], dtype=Float32, value=[...], name=base_model.classifier.original_module.dense.weight), requires_grad=True),
 Parameter (Tensor(shape=[1024], dtype=Float32, value=[...], name=base_model.classifier.original_module.dense.bias), requires_grad=True),
 Parameter (Tensor(shape=[2, 1024], dtype=Float32, value=[...], name=base_model.classifier.original_module.out_proj.weight), requires_grad=True),
 Parameter (Tensor(shape=[2], dtype=Float32, value=[0. 0.], name=base_model.classifier.original_module.out_proj.bias), requires_grad=True),
 Parameter (Tensor(shape=[1024, 1024], dtype=Float32, value=[...], name=base_model.classifier.modules_to_save.default.dense.weight), requires_grad=True),
 Parameter (Tensor(shape=[1024], dtype=Float32, value=[...], name=base_model.classifier.modules_to_save.default.dense.bias), requires_grad=True),
 Parameter (Tensor(shape=[2, 1024], dtype=Float32, value=[...], name=base_model.classifier.modules_to_save.default.out_proj.weight

In [13]:
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

grad_fn = mindspore.value_and_grad(forward_fn, None, model.trainable_params())

def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer(grads)
    return loss

for epoch in range(num_epochs):
    model.set_train()
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        loss = train_step(**batch)
        lr_scheduler.step()

    model.set_train(False)
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        outputs = model(**batch)
        predictions = outputs.logits.argmax(axis=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

100%|███████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  5.37it/s]


epoch 0: {'accuracy': 0.6446078431372549, 'f1': 0.7377938517179023}


100%|███████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  6.33it/s]


epoch 1: {'accuracy': 0.7254901960784313, 'f1': 0.8181818181818181}


100%|███████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  6.17it/s]


...

epoch 17: {'accuracy': 0.7254901960784313, 'f1': 0.8308157099697885}


100%|███████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  6.17it/s]


epoch 18: {'accuracy': 0.7156862745098039, 'f1': 0.8237082066869301}


100%|███████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  6.20it/s]

epoch 19: {'accuracy': 0.7156862745098039, 'f1': 0.8237082066869301}
